## API Call Pattern

In [ ]:
import os
from urllib.parse import quote
import requests
from dotenv import load_dotenv

load_dotenv()
TOKEN = os.getenv("SCANSAN_API_KEY")
if not TOKEN:
    raise ValueError("SCANSAN_API_KEY not found in environment variables.")

BASE_URL = "https://api.scansan.com/v1"

# function below is used as a GET helper, which builds full URL, adds auth header and raises useful errors in case of request failure
def get_json(path: str) -> dict:

    url = BASE_URL + path
    headers = {"X-Auth-Token": TOKEN}
    r = requests.get(url, headers=headers, timeout=30)

    # try parsing JSON even for request errors (often includes message)
    try:
        payload = r.json()
    except Exception:
        payload = {"raw_text": r.text}

    if not r.ok:
        raise RuntimeError(f"HTTP {r.status_code} for {url}: {payload}")
    return payload

# function below will encode the space commonly found in most postcodes
def encode_postcode(postcode: str) -> str:
 # quote will encode the space -> %20
 return quote(postcode.strip().upper())

## fetch functions for current and historical.... returns json files holding data from both API requests

In [ ]:
def fetch_current(postcode: str) -> dict:

    pc = encode_postcode(postcode)
    return get_json(f"/postcode/{pc}/valuations/current")

def fetch_historical(postcode: str) -> dict:
    pc = encode_postcode(postcode)
    return get_json(f"/postcode/{pc}/valuations/historical")


## cleaner function for current valuations of property in selected postcode

In [ ]:
import pandas as pd
import numpy as np

def clean_current(payload: dict) -> pd.DataFrame:
    rows = payload.get("data", []) or []
    df = pd.DataFrame(rows)

    if df.empty:
        return df

    # 1) Address is your join key later
    df["property_address"] = df["property_address"].astype(str).str.strip()

    # 2) Split bounded_valuation [low, high] into columns
    def get_low(x):
        return x[0] if isinstance(x, list) and len(x) > 0 else np.nan

    def get_high(x):
        return x[1] if isinstance(x, list) and len(x) > 1 else np.nan

    df["valuation_low"] = df["bounded_valuation"].apply(get_low)
    df["valuation_high"] = df["bounded_valuation"].apply(get_high)
    df["valuation_mid"] = (df["valuation_low"] + df["valuation_high"]) / 2

    # 3) Parse numeric/dates safely
    df["last_sold_price"] = pd.to_numeric(df["last_sold_price"], errors="coerce")
    df["last_sold_date"] = pd.to_datetime(df["last_sold_date"], errors="coerce")

    # 4) Nullable booleans -> False/True
    for col in ["lower_outlier", "upper_outlier"]:
        df[col] = df[col].fillna(False).astype(bool)

    # 5) Remove clearly broken rows (no address)
    df = df[df["property_address"].notna() & (df["property_address"] != "")].copy()

    return df


## cleaner function for HISTORICAL valuations of property in selected postcode

In [ ]:
def clean_historical(payload: dict) -> pd.DataFrame:
    rows = payload.get("data", []) or []
    out = []

    for item in rows:
        addr = item.get("property_address")
        valuations = item.get("valuations", []) or []

        # explode the nested list into many rows
        for v in valuations:
            out.append({
                "property_address": addr,
                "date": v.get("date"),
                "valuation": v.get("valuation")
            })

    df = pd.DataFrame(out)
    if df.empty:
        return df

    df["property_address"] = df["property_address"].astype(str).str.strip()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["valuation"] = pd.to_numeric(df["valuation"], errors="coerce")

    # drop invalid observations
    df = df.dropna(subset=["property_address", "date", "valuation"])
    df = df[df["valuation"] > 0].copy()

    # remove duplicates if any
    df = df.drop_duplicates(subset=["property_address", "date"], keep="last")

    return df.sort_values(["property_address", "date"]).reset_index(drop=True)
